In [1]:
# something is wrong with 2.4.4 and java 8. Should use lower version. Also only java 8 works
!pip install pyspark==2.4.4

In [2]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


In [3]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2


In [4]:
!java -version 

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf /content/spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
from pyspark import SparkConf, SparkContext
import collections
conf = SparkConf().setMaster("local").setAppName("MovieRental")
# need sc on cluster

sc = SparkContext(conf = conf)

In [41]:
lines = sc.textFile("payment.csv")
header = lines.zipWithIndex().filter(lambda row_index: row_index[1] > 0).keys()
header.first()

'2,1,1,573,0.99,05/28/2005 10:35,02/15/2006 22:12'

**Question 1:**  Average amount of payment by month & year. 

In [22]:
# from pyspark import SparkConf, SparkContext


# this function gets month, year and adjusted closed price
def parseLine(line):
    fields = line.split(',')
    datetimedata = fields[5]
    datetimearray = datetimedata.split('/')
    datetimearray_ = datetimearray[2].split(' ') 
    monthyear = datetimearray[0]+datetimearray_[0]
    amount = float(fields[4])
    return (monthyear, amount)
  
  
header = sc.textFile("payment_3.csv")
header.take(3)

['1,1,1,76,2.99,05/25/2005 11:30,02/15/2006 22:12',
 '2,1,1,573,0.99,05/28/2005 10:35,02/15/2006 22:12',
 '3,1,1,1185,5.99,06/15/2005 00:54,02/15/2006 22:12']

In [23]:
rdd = header.map(parseLine)
rdd.take(3)

[('052005', 2.99), ('052005', 0.99), ('062005', 5.99)]

In [24]:
priceByMonthYear = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByMonthYear = priceByMonthYear.mapValues(lambda x: x[0] / x[1])
results = averagesByMonthYear.collect()
for result in results:
    print (result)

('052005', 4.169775280898756)
('062005', 4.166038062283568)
('072005', 4.22796751601904)
('082005', 4.232834534904548)
('022006', 2.8251648351648404)


**Question 2:** Total payment by customer id. 

In [28]:
def parse_custid(line):
    fields = line.split(',')
    custid = fields[1]
    amount = float(fields[4])
    return (custid, amount)

rdd = header.map(parse_custid)
totalpmt = rdd.reduceByKey(lambda x,y : x + y)
results = totalpmt.collect()

for result in results:
    print (result)


('1', 118.67999999999992)
('2', 128.72999999999993)
('3', 135.73999999999998)
('4', 81.78)
('5', 144.61999999999998)
('6', 93.71999999999998)
('7', 151.67)
('8', 92.75999999999999)
('9', 89.76999999999998)
('10', 99.74999999999997)
('11', 106.75999999999996)
('12', 103.71999999999998)
('13', 131.72999999999996)
('14', 117.71999999999994)
('15', 134.67999999999995)
('16', 120.70999999999995)
('17', 98.78999999999998)
('18', 91.77999999999999)
('19', 125.75999999999998)
('20', 115.69999999999995)
('21', 155.65)
('22', 113.77999999999999)
('23', 119.69999999999996)
('24', 95.74999999999999)
('25', 115.70999999999995)
('26', 152.65999999999997)
('27', 126.68999999999996)
('28', 111.67999999999995)
('29', 140.63999999999996)
('30', 123.65999999999993)
('31', 104.73999999999997)
('32', 127.70999999999995)
('33', 91.78999999999998)
('34', 89.76)
('35', 129.67999999999995)
('36', 92.72999999999999)
('37', 95.76999999999997)
('38', 127.65999999999993)
('39', 141.70999999999995)
('40', 110.72999

**Question 3:**  Total payment by staff id. 


In [30]:
def parse_staffid(line):
    fields = line.split(',')
    staffid = fields[2]
    amount = float(fields[4])
    return (staffid, amount)

rdd = header.map(parse_staffid)
totalpmt = rdd.reduceByKey(lambda x,y : x + y)
results = totalpmt.collect()

for result in results:
    print (result)


('1', 33489.470000005174)
('2', 33927.04000000482)


 **Question 4:**  Average payment by month & year and by customer id.



In [0]:
def parseLine_custid(line):
    fields = line.split(',')
    datetimedata = fields[5]
    datetimearray = datetimedata.split('/')
    datetimearray_ = datetimearray[2].split(' ') 
    monthyear_custid = datetimearray[0]+datetimearray_[0] + ' , '+ fields[1] 
    amount = float(fields[4])
    return (monthyear_custid, amount)
  

In [40]:
rdd = header.map(parseLine_custid)
priceByMonthYear_cust = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByMonthYear_custid = priceByMonthYear_cust.mapValues(lambda x: x[0] / x[1])
results = averagesByMonthYear_custid.collect()
for result in results:
    print (result)

('052005 , 1', 1.9900000000000002)
('062005 , 1', 4.561428571428571)
('072005 , 1', 4.240000000000001)
('082005 , 1', 2.899090909090909)
('052005 , 2', 4.99)
('062005 , 2', 2.99)
('072005 , 2', 5.418571428571428)
('082005 , 2', 4.080909090909092)
('052005 , 3', 2.49)
('062005 , 3', 6.49)
('072005 , 3', 5.2976923076923095)
('082005 , 3', 5.1328571428571435)
('062005 , 4', 2.8233333333333337)
('072005 , 4', 4.390000000000001)
('082005 , 4', 3.89909090909091)
('052005 , 5', 3.3233333333333337)
('062005 , 5', 3.9900000000000007)
('072005 , 5', 3.740000000000001)
('082005 , 5', 4.143846153846155)
('022006 , 5', 0.99)
('052005 , 6', 2.99)
('062005 , 6', 2.24)
('072005 , 6', 2.99)
('082005 , 6', 4.073333333333334)
('052005 , 7', 4.19)
('062005 , 7', 2.1900000000000004)
('072005 , 7', 5.444545454545456)
('082005 , 7', 4.990000000000001)
('052005 , 8', 6.99)
('062005 , 8', 4.656666666666667)
('072005 , 8', 2.9900000000000007)
('082005 , 8', 4.49)
('052005 , 9', 3.6566666666666667)
('062005 , 9'

(('200507', '1'), 4.240000000000001)